<a href="https://colab.research.google.com/github/sundarp17/semi-supervised/blob/main/train6_svc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:

train6 = pd.read_csv(r'/content/train_data6.csv')
test = pd.read_csv(r'/content/test_data.csv')

In [3]:

train6['Target'] = train6['Target'].replace(['Others'],'Invalid')


In [4]:
train6['Target'].unique()

array(['Invalid', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [5]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)

train6_Xtfidf = tfidf_vect.fit_transform(train6['Sentence'])


In [7]:
test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))

In [8]:
unlabel_1 = test.loc[:100]
print("length of unlabel_1",len(unlabel_1))
unlabel_2 = test.loc[101:200]
print("length of unlabel_2",len(unlabel_2))
unlabel_3 = test.loc[201:300]
print("length of unlabel_3",len(unlabel_3))
unlabel_4 = test.loc[301:400]
print("length of unlabel_4",len(unlabel_4))
unlabel_5 = test.loc[401:]
print("length of unlabel_5",len(unlabel_5))

length of unlabel_1 101
length of unlabel_2 100
length of unlabel_3 100
length of unlabel_4 100
length of unlabel_5 120


In [9]:
unlabel_2.reset_index(inplace=True)
del unlabel_2['index']
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
4,906,agreed accompany gave pawn ticket money reclai...,Facts


In [10]:
unlabel_3.reset_index(inplace=True)
del unlabel_3['index']
unlabel_3.head()

,Unnamed: 0,Sentence,Target
0,1882,find evidence amply sufficient sustain juryâs ...,Analysis
1,964,actions show thatâ,Invalid
2,1417,certification attending physician shown filed ...,Facts
3,2932,testified agent bland first left automobile wa...,Facts
4,1584,officer testified could smell odor alcohol app...,Facts


In [11]:
unlabel_4.reset_index(inplace=True)
del unlabel_4['index']
unlabel_4.head()

,Unnamed: 0,Sentence,Target
0,266,section 6 pl,Invalid
1,3469,amended answer appellant alleged bond invalid ...,Analysis
2,1471,evidence sufficient support conviction appella...,Analysis
3,1990,cases cited note 19,Invalid
4,2757,july 8 1960 order entered revoking said probat...,Facts


In [12]:
unlabel_5.reset_index(inplace=True)
del unlabel_5['index']
unlabel_5.head()

,Unnamed: 0,Sentence,Target
0,2979,overrule appellantâs contention court erred pe...,Analysis
1,2135,state regarded authority case,Invalid
2,1591,testimony adduced appellant mother lillie mae ...,Facts
3,658,appellant needle marks inside left elbow,Facts
4,2562,analysis contents 3 capsules chemist kenneth a...,Facts


In [13]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.calibration import CalibratedClassifierCV


X_train, x_val, Y_train, y_val = train_test_split(train6_Xtfidf,train6['Target'],test_size=0.26,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model=calibrator.fit(X_train, Y_train)
pred_svm = model.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6980088495575221
                  precision    recall  f1-score   support

        Analysis       0.53      0.40      0.46       121
      Conclusion       0.84      0.56      0.68        48
           Facts       0.75      0.92      0.83       494
         Invalid       0.53      0.46      0.50       127
           Issue       0.59      0.45      0.51        49
Rule/Law/Holding       0.66      0.29      0.40        65

        accuracy                           0.70       904
       macro avg       0.65      0.52      0.56       904
    weighted avg       0.68      0.70      0.68       904



In [14]:
x_un1 = tfidf_vect.transform(unlabel_1['Sentence'])
pred_unlabel_1 = model.predict_proba(x_un1)

In [15]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_1:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 2, 2, 2, 2, 2, 2, 3]
[0.9949578574861346, 0.9967987895462261, 0.9999088552229572, 0.9989651470349759, 0.9963649650961459, 0.9942613210883398, 0.9822437583486943, 0.9994536693852597, 0.9981741857762731, 0.9974416989747814, 0.9828031550789116, 0.9996190859369667, 0.9916713636775566, 0.9852635318540467, 0.9917464868103523, 0.9961745607419876, 0.9960039503152831, 0.9839332300545885, 0.9992953776406581, 0.9997502104004056, 0.9872353079886922, 0.996427296513276, 0.9897348217319717, 0.9866554932126123, 0.9876920584498062, 0.9955487869696726, 0.9872615583846382, 0.998605655680586, 0.9989531081501208, 0.98379556994977, 0.9942126860971225, 0.9995627392842017, 0.9907605596868696, 0.9896143669654421, 0.9898504386745967, 0.9908230716092312, 0.9991452979782945, 0.9992510993659963, 0.9985536016096686, 0.995142095969377]
[0, 4, 5, 11, 12, 17, 18, 23, 24, 29, 30, 31, 33, 36, 37, 39, 40, 41, 45, 48, 49, 5

In [16]:
unlabel_1=unlabel_1.loc[[0, 4, 5, 11, 12, 17, 18, 23, 24, 29, 30, 31, 33, 36, 37, 39, 40, 41, 45, 48, 49, 50, 51, 52, 55, 56, 59, 60, 62, 67, 70, 76, 78, 79, 84, 87, 94, 95, 96, 98],:]
unlabel_1.head()

,Unnamed: 0,Sentence,Target
0,2173,darden sr testified âœi saw staggering â prett...,Facts
4,79,eligible leaves renew contacts family,Facts
5,1955,got downstairs ponce went car wife left home,Facts
11,1604,police notified deceased found lying mrs moers...,Facts
12,1102,shown vine street drug storeâs supply narcotic...,Facts


In [17]:
frame_1 = [train6,unlabel_1]
train6_1 = pd.concat(frame_1)
len(train6_1)

3516

In [18]:
x_train_1 = tfidf_vect.transform(train6_1['Sentence'])
x_train_1.shape
X_train, x_val, Y_train, y_val = train_test_split(x_train_1,train6_1['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_1=calibrator.fit(X_train, Y_train)
pred_svm = model_1.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6846590909090909
                  precision    recall  f1-score   support

        Analysis       0.53      0.49      0.51       100
      Conclusion       0.76      0.59      0.67        37
           Facts       0.75      0.90      0.81       355
         Invalid       0.55      0.51      0.53       109
           Issue       0.71      0.50      0.59        44
Rule/Law/Holding       0.64      0.24      0.35        59

        accuracy                           0.68       704
       macro avg       0.66      0.54      0.58       704
    weighted avg       0.67      0.68      0.67       704



In [19]:
x_un2 = tfidf_vect.transform(unlabel_2['Sentence'])
pred_unlabel_2 = model_1.predict_proba(x_un2)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_2:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2]
[0.9964384086246282, 0.9811207287041884, 0.9995013257699671, 0.9887023962951773, 0.9923563254023747, 0.9976051516008968, 0.9999368651547732, 0.9867423732033807, 0.9994301519128396, 0.9883980201679434, 0.9985455542345231, 0.9941750324842581, 0.996017393916734, 0.9996738678688784, 0.999943035329041, 0.9992008227502318, 0.9999362191067888, 0.9995097123757043, 0.9984063764417104, 0.9964444384040979, 0.9994735483747152, 0.9995220014636818, 0.9998622305001157, 0.9991362349724996, 0.9954515832950832, 0.9971156608931537, 0.9961982646281509, 0.9917384931799431, 0.999499170348162, 0.995979310779715, 0.990682239074748, 0.999461460771576, 0.9994582719808293, 0.982984932039937, 0.9884163513656504]
[0, 1, 2, 4, 6, 7, 12, 13, 20, 21, 29, 31, 32, 36, 37, 39, 42, 49, 54, 55, 59, 63, 64, 69, 70, 71, 75, 77, 83, 85, 86, 88, 91, 95, 99]
35
35
35


In [20]:
unlabel_2 = unlabel_2.loc[[0, 1, 2, 4, 6, 7, 12, 13, 20, 21, 29, 31, 32, 36, 37, 39, 42, 49, 54, 55, 59, 63, 64, 69, 70, 71, 75, 77, 83, 85, 86, 88, 91, 95, 99],:]
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
4,906,agreed accompany gave pawn ticket money reclai...,Facts
6,570,appellant performed work asked 7500 money ther...,Facts


In [21]:
frame_2 = [train6_1,unlabel_2]
train6_2 = pd.concat(frame_2)
len(train6_2)

3551

In [22]:
x_train_2 = tfidf_vect.transform(train6_2['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_2,train6_2['Target'],test_size=0.27,random_state=2)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_2=calibrator.fit(X_train, Y_train)
pred_svm = model_2.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6579770594369134
                  precision    recall  f1-score   support

        Analysis       0.48      0.41      0.44       145
      Conclusion       0.81      0.45      0.58        47
           Facts       0.73      0.87      0.80       496
         Invalid       0.51      0.48      0.50       159
           Issue       0.63      0.43      0.51        56
Rule/Law/Holding       0.56      0.32      0.41        56

        accuracy                           0.66       959
       macro avg       0.62      0.49      0.54       959
    weighted avg       0.65      0.66      0.64       959



In [23]:
x_un3 = tfidf_vect.transform(unlabel_3['Sentence'])
pred_unlabel_3 = model_2.predict_proba(x_un3)

pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_3:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 3, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[0.9932201811361425, 0.9934681919409485, 0.9965858819135535, 0.9930820551518224, 0.9990264446253737, 0.9993680657204873, 0.9995539372685107, 0.9849677551107988, 0.9847364264139008, 0.9988400376169039, 0.9931097495253124, 0.9858613110297548, 0.999741563312755, 0.984988967484146, 0.9936519943505994, 0.9986403584602913, 0.9816597687322136, 0.999422386176518, 0.9951568520952124, 0.9925803667770879, 0.995803081466059, 0.9984240892336346, 0.9910121217122034, 0.9864209901024868, 0.9803944488933175, 0.9967682336197994, 0.9921229479878463, 0.9982657162627759, 0.9958147510945422, 0.9957181063689154, 0.9997845813029709, 0.9922673163032504, 0.9994064238344406, 0.99849544372453, 0.9997630473335631, 0.9979071044953841, 0.9919379976389204, 0.9995339261380839, 0.9985676571773399, 0.985307550912728, 0.9980326537350982, 0.9977263089791167, 0.9970663288167679, 0.99705494253

In [24]:
unlabel_3=unlabel_3.loc[[0, 2, 3, 6, 7, 8, 12, 15, 17, 20, 23, 26, 27, 28, 29, 30, 32, 35, 36, 40, 43, 46, 47, 48, 51, 52, 53, 54, 56, 57, 63, 64, 65, 67, 68, 70, 71, 72, 75, 78, 79, 89, 96, 97],:]
frame_3 = [train6_2,unlabel_3]
train6_3 = pd.concat(frame_3)

x_train_3 = tfidf_vect.transform(train6_3['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_3,train6_3['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_3=calibrator.fit(X_train, Y_train)
pred_svm = model_3.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6828929068150209
                  precision    recall  f1-score   support

        Analysis       0.42      0.38      0.40        84
      Conclusion       0.76      0.52      0.62        42
           Facts       0.76      0.89      0.82       383
         Invalid       0.54      0.44      0.48       128
           Issue       0.82      0.56      0.67        41
Rule/Law/Holding       0.50      0.39      0.44        41

        accuracy                           0.68       719
       macro avg       0.63      0.53      0.57       719
    weighted avg       0.67      0.68      0.67       719



In [25]:
x_un4 = tfidf_vect.transform(unlabel_4['Sentence'])
pred_unlabel_4 = model_3.predict_proba(x_un4)


pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_4:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 5, 2, 2]
[0.9990947410394317, 0.9997028135715613, 0.9961544973498928, 0.9928001637531226, 0.999956981643511, 0.9935218407692799, 0.9907308116156173, 0.9984147463039694, 0.9940648834367304, 0.9996958284919519, 0.9969824670973315, 0.9972819464024668, 0.9986406687207936, 0.9964232517317747, 0.9998322643101465, 0.9965676400758385, 0.9968180989147207, 0.9998487481694136, 0.9984081646879166, 0.9992694036040036, 0.9969319409409964, 0.9833317289562051, 0.999514186875045, 0.9936702628487959, 0.9977570342936068, 0.9978507693559999, 0.9966649606755151, 0.9921840305166133, 0.9910727500345619, 0.9842823207764833, 0.9977631493641617, 0.9975204488011132, 0.9964746599218056, 0.9950195989155951, 0.9999734788282102, 0.9934824324738506, 0.9911716574747486, 0.9897984582056393, 0.9943084810859806, 0.9820605436751467, 0.9990639674569797, 0.9899330352515503, 0.9986582556547122, 0.992163

In [26]:
unlabel_4=unlabel_4.loc[[2, 4, 7, 8, 10, 11, 12, 15, 16, 18, 21, 22, 27, 28, 29, 30, 31, 32, 34, 37, 38, 39, 40, 49, 50, 56, 58, 60, 61, 64, 67, 69, 70, 71, 75, 80, 82, 85, 87, 88, 93, 94, 97, 99],:]

In [27]:
frame_4 = [train6_3,unlabel_4]
train6_4 = pd.concat(frame_4)
len(train6_4)
x_train_4 = tfidf_vect.transform(train6_4['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_4,train6_4['Target'],test_size=0.27,random_state=42)
support = svm.LinearSVC()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_4=calibrator.fit(X_train, Y_train)
pred_svm = model_4.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6775178026449644
                  precision    recall  f1-score   support

        Analysis       0.45      0.46      0.46       133
      Conclusion       0.64      0.58      0.61        48
           Facts       0.79      0.87      0.82       530
         Invalid       0.49      0.45      0.47       148
           Issue       0.59      0.41      0.48        58
Rule/Law/Holding       0.64      0.41      0.50        66

        accuracy                           0.68       983
       macro avg       0.60      0.53      0.56       983
    weighted avg       0.67      0.68      0.67       983



In [28]:
x_un5 = tfidf_vect.transform(unlabel_5['Sentence'])

pred_unlabel_5 = model_4.predict_proba(x_un5)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_5:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 5, 3, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 3, 3, 0]
[0.9993112463009013, 0.9881098194815393, 0.9901971608792257, 0.9947826621898405, 0.9961940508431238, 0.996661879916227, 0.9852120805731471, 0.9969593709947147, 0.9994725418885259, 0.99952401790585, 0.9943803484276718, 0.9913289500167763, 0.982519106785029, 0.9997475651059429, 0.9941861214521452, 0.9993723367204942, 0.9986460335139431, 0.9890675399121425, 0.9881831086764552, 0.9989306351908325, 0.9973145889153587, 0.9983189316944261, 0.9890409082073843, 0.991514061354604, 0.9955756485805142, 0.9843484205917578, 0.9847485038166551, 0.9811370877168798, 0.9998924712685616, 0.9989669876583865, 0.9951311840331472, 0.9954044222931201, 0.9983511944023546, 0.9985942627912299, 0.9869489503211928, 0.9944390614032795, 0.9901069533986424, 0.9921478896105204, 0.9913245655419506, 0.9884247089321849]
[3, 4, 7, 12, 18, 19, 20, 22, 23, 26, 27, 32, 35, 36, 46, 52, 53, 56, 59, 60, 63, 6

In [29]:
unlabel_5 = unlabel_5.loc[[3, 4, 7, 12, 18, 19, 20, 22, 23, 26, 27, 32, 35, 36, 46, 52, 53, 56, 59, 60, 63, 65, 66, 67, 71, 73, 75, 78, 82, 86, 91, 93, 94, 95, 100, 103, 104, 106, 107, 110],:]

In [30]:
frame_5 = [train6_4,unlabel_5]
train6_5 = pd.concat(frame_5)
len(train6_5)
x_train_5 = tfidf_vect.transform(train6_5['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_5,train6_5['Target'],test_size=0.27,random_state=42)
support = svm.LinearSVC()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_5=calibrator.fit(X_train, Y_train)
pred_svm = model_5.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6881287726358148
                  precision    recall  f1-score   support

        Analysis       0.50      0.44      0.46       142
      Conclusion       0.69      0.56      0.62        55
           Facts       0.80      0.86      0.83       533
         Invalid       0.45      0.51      0.47       130
           Issue       0.68      0.60      0.64        57
Rule/Law/Holding       0.61      0.40      0.48        77

        accuracy                           0.69       994
       macro avg       0.62      0.56      0.58       994
    weighted avg       0.68      0.69      0.68       994



In [31]:
t_p = tfidf_vect.transform(test['Sentence'])
test_pred = model_5.predict(t_p)
print('Accuracy %s' % accuracy_score(test_pred,test['Target']))

Accuracy 0.6737044145873321


In [32]:
from sklearn.metrics import precision_recall_fscore_support
print(precision_recall_fscore_support(test['Target'],test_pred,average='macro'))

(0.6518059180187226, 0.5854681395496341, 0.6076750578298377, None)
